# Generating the electronic Hamiltonian

For chemistry applications in quantum computing, one needs first to design the electronic Hamiltonian and convert it to qubit Hamiltonian. In this tutorial, we will explain how user can generate the electronic Hamiltonian for their subsequent quantum computing simulation.

The electronic Hamiltonian describes the energy of electrons in the electrostatic field of nuclei that are assumed to be stationary with respect to an inertial frame [(1)](https://en.wikipedia.org/wiki/Molecular_Hamiltonian). This operator plays a central role in computational chemistry and physics for calculating properties of molecules including thermal conductivity, specific heat, electrical conductivity, optical properties, magnetic properties, and chemical reactivity.

In quantum mechanics generally, the Hamiltonian operator represents the total energy of a system, including both kinetic and potential energy components. The electronic Hamiltonian specifically focuses on the electronic component of this energy within molecular systems.

#### Second Quantized formulation.

The second quantized formulation expresses the Hamiltonian in terms of creation and annihilation operators that automatically preserve the antisymmetry requirements for fermions. This representation is particularly useful for implementing computational methods in quantum chemistry.

$$H = \sum_{pq} h_{pq} a_p^\dagger a_q + \frac{1}{2} \sum_{pqrs} g_{pqrs} a_p^\dagger a_q^\dagger a_r a_s$$

where:
- $h_{pq}$ are the one-electron integrals
- $g_{pqrs}$ are the two-electron integrals
- $a_p^\dagger$ and $a_p$ are the fermion creation and annihilation operators

This formulation provides significant advantages for computational implementations and development of electron correlation methods.

### Computational Implementation

The computational implementation of electronic Hamiltonians involves several key steps that transform the abstract operator into a concrete numerical representation.

- Basis Set Expansion: The first step is to express the electronic Hamiltonian in a finite basis of atomic orbitals (AOs). These are typically Gaussian-type orbitals centered on atoms, chosen for computational efficiency
- Integral Calculation: Once a basis is chosen, one-electron and two-electron integrals are computed in the AO basis. These represent matrix elements of the Hamiltonian components between different basis functions
- Self-Consistent Field Calculation: A self-consistent field (SCF) procedure, typically using the Hartree-Fock method, is performed to obtain molecular orbitals (MOs) as linear combinations of AOs
- Integral Transformation: The AO integrals are transformed to the MO basis using the coefficients from the SCF calculation. This transformation is computationally intensive but crucial for correlated methods
- Spin Adaptation: For many applications, the spatial MO integrals are converted to spin MO integrals to account for electron spin explicitly

In this tutorial, we employ [pyscf](https://github.com/pyscf/pyscf) to perform these calculations of electron integrals.

In [ ]:
# Requires pyscf to be installed
%pip install pyscf

To calculate the electronic integrals, and generate the spin molecular Hamiltonian, import the following

In [1]:
from qchem.classical_pyscf import get_mol_hamiltonian

To convert from fermionic Hamiltonian to qubit hamiltonian, import the following

In [2]:
from qchem.hamiltonian import jordan_wigner_fermion

In [3]:
# Import the cudaq module
import cudaq

# Set the traget
# Double precision is recommended for the best performance.
cudaq.set_target("nvidia", option = "fp64")


## Full space Hamiltonian

### (a) Generate the molecular Hamiltonian using Restricted Hartree Fock molecular orbitals

In [6]:
H2_mol = "qchem/H2.xyz"

# Run HF, ccsd and compute the spin molecular hamiltonian using the HF molecular orbitals.
molecular_data = get_mol_hamiltonian(xyz=H2_mol, spin=0, charge=0, basis='sto3g', ccsd=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
e_nn = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons: ', nelectrons)
print('Total number of spatial orbitals: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Nuclear repulsion energy: ', e_nn)
print('Fermionic hamiltonian: ', fermionic_hamiltonian)
print('Energies: ', energies)

print('-------------------------------')
print('Jordan-Wigner transformation')
print('-------------------------------')
# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, e_nn, tolerance = 1e-12)

print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)


overwrite output file: qchem/H2-pyscf.log
[pyscf] Total number of orbitals =  2
[pyscf] Total number of electrons =  2
[pyscf] HF energy =  -1.116651247411522
[pyscf] Total R-CCSD energy =  -1.1372635051456803
-------------------------------
Molecular data
-------------------------------
Number of electrons:  2
Total number of spatial orbitals:  2
Number of spin molecular orbitals:  4
Nuclear repulsion energy:  0.7131768341239892
Fermionic hamiltonian:  -1.252101177118797 a_0^† a_0 + -1.252101177118797 b_1^† b_1 + 0.16859356817552068 a_0^† a_0^† a_0 a_0 + 0.16859356817552068 b_1^† b_1^† b_1 b_1 + 0.16859356817552068 a_0^† b_1^† b_1 a_0 + 0.16859356817552068 b_1^† a_0^† a_0 b_1 + 1.734723475976807e-18 a_0^† a_0^† a_0 a_2 + 1.734723475976807e-18 b_1^† b_1^† b_1 b_3 + 1.734723475976807e-18 a_0^† b_1^† b_1 a_2 + 1.734723475976807e-18 b_1^† a_0^† a_0 b_3 + 2.7755575615628914e-17 a_0^† a_0^† a_2 a_0 + 2.7755575615628914e-17 b_1^† b_1^† b_3 b_1 + 2.7755575615628914e-17 a_0^† b_1^† b_3 a_0 + 2

### (b) Generate the molecular Hamiltonian using Unrestricted Hartree Fock molecular orbitals

In [7]:
H2_mol = "qchem/H2.xyz"

# Run HF, ccsd and compute the spin molecular hamiltonian using the HF molecular orbitals.
molecular_data = get_mol_hamiltonian(xyz=H2_mol, spin=0, charge=0, basis='sto3g', UR=True, ccsd=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
e_nn = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons: ', nelectrons)
print('Total number of spatial orbitals: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Nuclear repulsion energy: ', e_nn)
print('Fermionic hamiltonian: ', fermionic_hamiltonian)
print('Energies: ', energies)

print('-------------------------------')
print('Jordan-Wigner transformation')
print('-------------------------------')
# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, e_nn, tolerance = 1e-12)

print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)

overwrite output file: qchem/H2-pyscf.log
[pyscf] Total number of alpha molecular orbitals =  2
[pyscf] Total number of beta molecular orbitals =  2
[pyscf] Total number of electrons =  2
[pyscf] HF energy =  -1.1166512474115222
[pyscf] Total UR-CCSD energy =  -1.1372635051456819
-------------------------------
Molecular data
-------------------------------
Number of electrons:  2
Total number of spatial orbitals:  2
Number of spin molecular orbitals:  4
Nuclear repulsion energy:  0.7131768341239892
Fermionic hamiltonian:  (-1.252101177118797+0j) a_0^† a_0 + (-1.252101177118797+0j) b_1^† b_1 + (-4.738989351833402e-16+0j) a_0^† a_2 + (-4.738989351833402e-16+0j) b_1^† b_3 + (-1.9852838903630502e-16+0j) a_2^† a_0 + (-1.9852838903630502e-16+0j) b_3^† b_1 + (-0.47632466628305814+0j) a_2^† a_2 + (-0.47632466628305814+0j) b_3^† b_3 + (0.1685935681755207+0j) a_0^† a_0^† a_0 a_0 + (0.1685935681755207+0j) b_1^† b_1^† b_1 b_1 + (0.1685935681755207+0j) a_0^† b_1^† b_1 a_0 + (0.1685935681755207+0j)

## Active space Hamiltonian:

The active space Hamiltonian [(2)](https://en.wikipedia.org/wiki/Complete_active_space) is a reduced-dimensionality quantum chemical model that focuses computational resources on the most chemically relevant orbitals and electrons. This approach enables practical calculations for systems with strong electron correlation by partitioning orbitals into three classes:

- Core orbitals: Fully occupied and treated as inert
- Active orbitals: Partially occupied, explicitly correlated
- Virtual orbitals: Unoccupied and excluded

The active space Hamiltonian $H^{AS}$ derives from the full electronic Hamiltonian through orbital space partitioning:

$$H^{AS} = \sum_{pq}^{\text{active}} \tilde{h}_{pq} \hat{a}_p^{\dagger} \hat{a}_q + \frac{1}{2} \sum_{pqrs}^{\text{active}} g_{pqrs} \hat{a}_p^{\dagger} \hat{a}_q^{\dagger} \hat{a}_r \hat{a}_s + E_{\text{core}}$$

Where:
- $\tilde{h}_{pq}$ are modified one-electron integrals incorporating mean-field effects from core orbitals
- $g_{pqrs}$ are two-electron integrals within the active space
- $E_{\text{core}}$ accounts for frozen core contributions



### (a) Generate the active space hamiltonian using RHF molecular orbitals.

In [8]:
H2O_mol = "qchem/H2O.xyz"

# Run HF, ccsd, casci and compute the active space spin molecular hamiltonian
# using the HF molecular orbitals with 4 electron and 4 orbitals in the active space.
molecular_data = get_mol_hamiltonian(xyz=H2O_mol, spin=0, charge=0, basis='631g', nele_cas=4, norb_cas=4, 
                                     ccsd=True, casci=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
ecore = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons in the active space: ', nelectrons)
print('Number of spatial orbitals in the active space: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Core energy: ', ecore)
print('CASCI Energy: ', energies['casci'])
#print('Fermionic hamiltonian: ', fermionic_hamiltonian)

print('-------------------------------')
print('Jordan-Wigner transformation')
print('-------------------------------')
# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, ecore)
print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)

overwrite output file: qchem/H2O-pyscf.log
[pyscf] Total number of orbitals =  13
[pyscf] Total number of electrons =  10
[pyscf] HF energy =  -75.98397553727901
[pyscf] R-CASCI energy using molecular orbitals=  -75.98508981028928
[pyscf] R-CCSD energy of the active space using molecular orbitals=  -75.98508980454689
-------------------------------
Molecular data
-------------------------------
Number of electrons in the active space:  4
Number of spatial orbitals in the active space:  4
Number of spin molecular orbitals:  8
Core energy:  -69.78937499839158
CASCI Energy:  -75.98508981028928
-------------------------------
Jordan-Wigner transformation
-------------------------------
[Cudaq] Qubit hamiltonian:  (-74.3886+0i) + (0.429698+0i) * Z0 + (0.429698+0i) * Z1 + (0.395076+0i) * Z2 + (0.395076+0i) * Z3 + (0.0246717+0i) * Z4 + (0.0246717+0i) * Z5 + (-0.0124873+0i) * Z6 + (-0.0124873+0i) * Z7 + (0.183927+0i) * Z0Z1 + (0.154956+0i) * Z0Z2 + (0.1664+0i) * Z0Z3 + (0.0441992+0i) * X0Z1Z2Z

### (b) Generate the active space Hamiltonian using the natural orbitals computed from MP2 simulation 

In [ ]:
H2O_mol = "qchem/H2O.xyz"

# Run HF, MP2, ccsd, casci and compute the active space spin molecular hamiltonian.
# Inspect the active space from the natural orbitals. Natural orbitals are computed using MP2. 
# The active space is defined by 4 electrons and 4 orbitals.
# The natural orbitals are used to compute the electron integrals.
molecular_data = get_mol_hamiltonian(xyz=H2O_mol, spin=0, charge=0, basis='631g',nele_cas=4, norb_cas=4, 
                                     MP2=True, natorb=True, ccsd=True, casci=True, integrals_natorb=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
ecore = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons in the active space: ', nelectrons)
print('Number of spatial orbitals in the active space: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Core energy: ', ecore)
#print('Fermionic hamiltonian: ', fermionic_hamiltonian)

print('-------------------------------')
print('Jordan-Wigner transformation')
print('-------------------------------')
# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, ecore)
print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)


overwrite output file: qchem/H2O-pyscf.log
[pyscf] Total number of orbitals =  13
[pyscf] Total number of electrons =  10
[pyscf] HF energy =  -75.98397553727898
[pyscf] R-MP2 energy=  -76.11280670065045
[pyscf] Natural orbital occupation number from R-MP2: 
[1.99995663e+00 1.99012740e+00 1.98197049e+00 1.97722983e+00
 1.97515174e+00 2.23585706e-02 2.09527832e-02 1.73621504e-02
 1.00838241e-02 2.32351484e-03 1.74875097e-03 4.27482714e-04
 3.06831462e-04]
[pyscf] R-CASCI energy using natural orbitals=  -76.03304212600106
[pyscf] R-CCSD energy of the active space using natural orbitals=  -76.03293564805296
-------------------------------
Molecular data
-------------------------------
Number of electrons in the active space:  4
Number of spatial orbitals in the active space:  4
Number of spin molecular orbitals:  8
Core energy:  -69.5920906407801
-------------------------------
Jordan-Wigner transformation
-------------------------------
[Cudaq] Qubit hamiltonian:  (-73.4465+0i) + (0.1985

### (c) Generate the active space Hamiltonian computed from the CASSCF molecular orbitals

In [ ]:
H2O_mol = "qchem/H2O.xyz"

# Run HF, MP2, casscf and compute the active space spin molecular hamiltonian.
# Inspect and the active space from the natural orbitals. Natural orbitals are computed using MP2. 
# The natural orbitals are used with casscf calaculations.
# The casscf  orbitals are used to compute the electron integrals.

molecular_data = get_mol_hamiltonian(xyz=H2O_mol, spin=0, charge=0, basis='631g', nele_cas=4, norb_cas=4, 
                                     MP2=True, natorb=True, casscf=True, integrals_casscf=True, verbose=True)

# If do not want to use the natural orbitals to inspect active space, set natorb=False (default).
#molecular_data = get_mol_hamiltonian(xyz=H2O_mol, spin=0, charge=0, basis='631g', nele_cas=4, norb_cas=4, 
#                                    casscf=True, integrals_casscf=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
ecore = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons in the active space: ', nelectrons)
print('Number of spatial orbitals in the active space: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Core energy: ', ecore)
#print('Fermionic hamiltonian: ', fermionic_hamiltonian)

print('-------------------------------')
print('Jordan-Wigner transformation')
print('-------------------------------')
# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, ecore)
print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)

overwrite output file: qchem/H2O-pyscf.log
[pyscf] Total number of orbitals =  13
[pyscf] Total number of electrons =  10
[pyscf] HF energy =  -75.983975537279
[pyscf] R-MP2 energy=  -76.11280670065047
[pyscf] Natural orbital occupation number from R-MP2: 
[1.99995663e+00 1.99012740e+00 1.98197049e+00 1.97722983e+00
 1.97515174e+00 2.23585706e-02 2.09527832e-02 1.73621504e-02
 1.00838241e-02 2.32351484e-03 1.74875097e-03 4.27482714e-04
 3.06831462e-04]
[pyscf] R-CASSCF energy using natural orbitals=  -76.03702384069328
-------------------------------
Molecular data
-------------------------------
Number of electrons in the active space:  4
Number of spatial orbitals in the active space:  4
Number of spin molecular orbitals:  8
Core energy:  -69.14810818511748
-------------------------------
Jordan-Wigner transformation
-------------------------------
[Cudaq] Qubit hamiltonian:  (-73.4246+0i) + (0.286+0i) * Z0 + (0.286+0i) * Z1 + (0.232955+0i) * Z2 + (0.232955+0i) * Z3 + (-0.262533+0i) 

### (d) Generate the electronic Hamiltonian using ROHF

In [11]:
geomerty = 'N 0.0 0.0 0.56; N 0.0 0.0 -0.56'
molecular_data = get_mol_hamiltonian(xyz= geomerty, spin=1, charge=1, basis='631g', nele_cas=5, norb_cas=6, ccsd=True, casci = True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
ecore = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons in the active space: ', nelectrons)
print('Number of spatial orbitals in the active space: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Core energy: ', ecore)

# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, ecore)
print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)

overwrite output file: N 0-pyscf.log
[pyscf] Total number of orbitals =  18
[pyscf] Total number of electrons =  13
[pyscf] HF energy =  -108.28029717676156
[pyscf] R-CASCI energy using molecular orbitals=  -108.37916901674107


WARN: RCCSD method does not support ROHF method. ROHF object is converted to UHF object and UCCSD method is called.


[pyscf] R-CCSD energy of the active space using molecular orbitals=  -108.37763087821682
-------------------------------
Molecular data
-------------------------------
Number of electrons in the active space:  5
Number of spatial orbitals in the active space:  6
Number of spin molecular orbitals:  12
Core energy:  -97.73731859734092
[Cudaq] Qubit hamiltonian:  (-106.644+0i) + (0.261983+0i) * Z0 + (0.261983+0i) * Z1 + (0.261983+0i) * Z2 + (0.261983+0i) * Z3 + (0.237545+0i) * Z4 + (0.237545+0i) * Z5 + (0.0601943+0i) * Z6 + (0.0601943+0i) * Z7 + (0.0601943+0i) * Z8 + (0.0601943+0i) * Z9 + (-0.199869+0i) * Z10 + (-0.199869+0i) * Z11 + (0.144151+0i) * Z0Z1 + (0.127483+0i) * Z0Z2 + (0.133039+0i) * Z0Z3 + (0.123982+0i) * Z0Z4 + (0.131017+0i) * Z0Z5 + (0.0936709+0i) * Z0Z6 + (0.134802+0i) * Z0Z7 + (0.120059+0i) * Z0Z8 + (0.125793+0i) * Z0Z9 + (0.0907631+0i) * Z0Z10 + (0.0949805+0i) * Z0Z11 + (0.133039+0i) * Z1Z2 + (0.127483+0i) * Z1Z3 + (0.131017+0i) * Z1Z4 + (0.123982+0i) * Z1Z5 + (0.134802+0

### (e) Generate electronic Hamiltonian using UHF

Note: in case of UHF only electronic Hamiltonian from HF molecular orbital are supported. Calculating the electron integrals from natural orbitals or CASSCF orbitals is not supported yet.

In [12]:
geomerty = 'N 0.0 0.0 0.56; N 0.0 0.0 -0.56'
molecular_data = get_mol_hamiltonian(xyz= geomerty, spin=1, charge=1, basis='631g', UR=True, nele_cas=5, norb_cas=6, ccsd=True, casci = True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
ecore = molecular_data[2]
nelectrons = molecular_data[3]
norbitals = molecular_data[4]
fermionic_hamiltonian = molecular_data[5]
energies = molecular_data[6]

print('-------------------------------')
print('Molecular data')
print('-------------------------------')
print('Number of electrons in the active space: ', nelectrons)
print('Number of spatial orbitals in the active space: ', norbitals)
print('Number of spin molecular orbitals: ', 2 * norbitals)
print('Core energy: ', ecore)
print(energies)

# Compute the qubit hamiltonian.
spin_ham = jordan_wigner_fermion(obi, tbi, ecore)
print('[Cudaq] Qubit hamiltonian: ', spin_ham)
print('[Cudaq] Number of terms in the Qubit hamiltonian: ', spin_ham.term_count)

overwrite output file: N 0-pyscf.log


WARN: system HOMO -0.376086973991372 >= system LUMO -0.376086973991372


[pyscf] Total number of alpha molecular orbitals =  18
[pyscf] Total number of beta molecular orbitals =  18
[pyscf] Total number of electrons =  13
[pyscf] HF energy =  -108.2893388837719
[pyscf] Converting nele_cas from 5 to (3, 2) (alpha, beta)
[pyscf] UR-CASCI energy using molecular orbitals=  -108.3877304184883
[pyscf] Converting nele_cas from 5 to (3, 2) (alpha, beta)
[pyscf] UR-CCSD energy of the active space using molecular orbitals=  -108.38620486132405
[pyscf] Converting nele_cas from 5 to (3, 2) (alpha, beta)
-------------------------------
Molecular data
-------------------------------
Number of electrons in the active space:  5
Number of spatial orbitals in the active space:  6
Number of spin molecular orbitals:  12
Core energy:  -103.1394501006166
{'hf': -108.2893388837719, 'mp2': None, 'casci': -108.3877304184883, 'casscf': None, 'ccsd': -108.38620486132405, 'fci': None}
[Cudaq] Qubit hamiltonian:  (-139.182+0i) + (2.65092+0i) * Z0 + (2.71224+0i) * Z1 + (2.72614+0i) * Z2